In [1]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import sproc,sum

import snowflake.snowpark
from snowflake.snowpark import functions as F

def initiateSession(): 
    connection_parameters = {
            "account": "rn65832.ap-southeast-1",
            "user": "  18",
            "password": "  @0409",
            "role": "Accountadmin", 
            "warehouse": "compute_wh",
            "database":"SNOWPARK_  ",
            "schema":"TEST_SCHEMA"
    }
    session = Session.builder.configs(connection_parameters).create()
    return session

session = initiateSession()

session.add_packages('snowflake-snowpark-python', 'pandas')

The version of package 'snowflake-snowpark-python' in the local environment is 1.13.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.


In [2]:

query = "create or replace stage snowpark_sproc_stage" +\
        " directory = (enable = true)" +\
        " copy_options = (on_error='skip_file')"
session.sql(query).collect()

[Row(status='Stage area SNOWPARK_SPROC_STAGE successfully created.')]

<H2>Create Stored Procedure Function</H2>

In [3]:
def df_transform_sproc(session: snowflake.snowpark.Session) -> str:
    
    df_orders = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.ORDERS") \
                .select(['O_ORDERKEY','O_CUSTKEY','O_TOTALPRICE']) \
                .filter("O_TOTALPRICE>=100000")

    df_lineitem = session.table("SNOWFLAKE_SAMPLE_DATA.TPCH_SF10.LINEITEM") \
                .select(['L_SUPPKEY','L_ORDERKEY','L_QUANTITY','L_PARTKEY'])

    df_order_lineitem = df_orders.join(df_lineitem,df_lineitem.L_ORDERKEY == df_orders.O_ORDERKEY) \
            .select(['L_ORDERKEY','O_CUSTKEY','O_TOTALPRICE','L_QUANTITY','L_SUPPKEY','L_PARTKEY'])
    
    try : 
        df_order_lineitem.write.mode("overwrite").save_as_table("order_lineitems")
        return 'success'
    except Exception as e : 
        return e

<H2>Create a Table in SF To Store the result of above Transformation</H2>

In [4]:
query = """create or replace table SNOWPARK_  .test_schema.order_lineitems 
        (
            L_ORDERKEY integer,O_CUSTKEY integer,O_TOTALPRICE float ,L_QUANTITY FLOAT,
            L_SUPPKEY integer,L_PARTKEY integer
        )
        """

session.sql(query).collect()

[Row(status='Table ORDER_LINEITEMS successfully created.')]

<h2>Deploy the Stored Proc</h2>

In [5]:
execute_dftransforms_sproc = sproc(df_transform_sproc,name='df_transform_sproc',stage_location='@snowpark_sproc_stage', \
                               is_permanent=True,replace=True)


<H2>Invoke Stored Proc</H2>

In [6]:
execute_dftransforms_sproc()

'success'